In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk import stem
stemmer = stem.PorterStemmer()
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string
punct = list(string.punctuation)
from collections import Counter
import requests
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
#!pip install PRAW
import numpy as np
import praw
import datetime



# The basics of lemmatising and stemming

In [ ]:
sent = "The quick brown fox jumped over the lazy dogs. The community of fowl sang in the dawn. The blind mice retreated before the feral felines. The geese flew north to escape the wolves"

In [ ]:
words = word_tokenize(sent)
stems = [stemmer.stem(i) for i in words]
lemmas = [lemmatizer.lemmatize(i) for i in words]

In [ ]:
with open('wuthering_heights.txt', 'r') as f:
    text = f.read()

In [ ]:
text = text.encode('ascii', 'ignore')
text = text.decode()


In [ ]:
text = ' '.join(text.splitlines())
text = text.lower()

In [ ]:
words = word_tokenize(text)

In [ ]:
words[:6]

In [ ]:
lemmas = [lemmatizer.lemmatize(i) for i in words]

In [ ]:
lemmas = [i for i in lemmas if i not in stops and i not in punct]

In [ ]:
counts = pd.Series(Counter(lemmas))

In [ ]:
counts

In [ ]:
sns.displot(counts, kind = 'kde')
plt.show()

In [ ]:
sns.lineplot(x = [len(i) for i in counts.index], y = [i for i in counts.values])

In [ ]:
counts.sort_values()

# Using APIs to obtain data

## [Tracking the UK's journey to carbon zero](https://data.climateemergency.uk/)

In [ ]:
URL = 'https://data.climateemergency.uk/api/searchterms/'

URL2 = "https://data.climateemergency.uk/api/commitments/"

PARAMS = {'min_results':1}

r = requests.get(url = URL, params = PARAMS)

r = r.json()


In [ ]:
r['results'][0]

## [The Project Gutenberg API](https://gutendex.com/books/)

In [ ]:
params_g = {'topic':'love'}

gut = 'https://gutendex.com/books/'

rg = r = requests.get(url = gut, params = params_g)


In [ ]:
rg.json()

## The Reddit API

1. Register for a Reddit account
2. Go to https://www.reddit.com/prefs/apps and create a script app.
3. Fill out the relevant fields, putting http://localhost:8080 for the URL fields
4. Record the client_id (top left) and secret. The user_agent field can be arbitrary.
5. The auth credentials are as follows: 
   `reddit = praw.Reddit(user_agent='YOURUSERAGENT',
   client_id='YOURCLIENTID', client_secret="SECRET",
   username='USERNAME', password='PASSWORD')`

In [ ]:
reddit = praw.Reddit(user_agent='VAD',
                     client_id='eCo_TWE_BA_zFA', client_secret="1gsqXgMrZBoQBVYf40hgtvMS_Ro",
                     username='textureai', password='7233190s')

In [ ]:
def submission(submission_id): ## submission_id can be URL or submission ID
    try:
        submission = reddit.submission(url = submission_id)
    except:
        submission = reddit.submission(submission_id)
    title = submission.title
    submission.comments.replace_more() ## loads new page if cooments are multipage
    text = [i.body for i in submission.comments]
    score = [i.score for i in submission.comments]
    user = [i.author for i in submission.comments]
    date = [datetime.datetime.fromtimestamp(i.created) for i in submission.comments]
    df = pd.DataFrame()
    df['text'] = text
    df['datetime'] = date
    df['score'] = score
    df['subreddit'] = submission.subreddit
    df['redditor'] = user
    df['type'] = 'comment'
    df['title'] = title
    df = df.sort_values('score', ascending = False).reset_index(drop = True)
    return df

In [ ]:
df = submission('https://www.reddit.com/r/AskReddit/comments/10orx1j/what_movies_soundtrack_is_an_absolute_banger/')

# Exercises

1. Write functions for interacting with the Reddit API that retrieve comments by (a) subreddit aand (b) by specific Reddit user and saves the result as a dataframe.
2. Identify a topic of interest to you and retrieve the top 1k comments on that subreddit.
3. Create a new column in your dataframe and that consists of the tokenized and lemmatized comments